In [1]:
from scipy.io import arff
import pandas as pd

In [2]:
# df = pd.read_csv('data/processed/ckd_full.csv')

In [3]:
patients_df = pd.read_csv('synthea_data/patients.csv')
obs_df = pd.read_csv('synthea_data/observations.csv')

# uncomment for all observation types
# set(pd.read_csv('synthea_data/observations.csv').DESCRIPTION.values)

In [5]:
kidney_obs = obs_df[obs_df['DESCRIPTION'] == 'Glomerular filtration rate/1.73 sq M.predicted']
kidney_obs = kidney_obs[['DATE', 'PATIENT', 'DESCRIPTION', 'VALUE', 'UNITS']].sort_values('DATE').drop_duplicates('PATIENT', keep='last')
gfr_df = pd.pivot(kidney_obs, index='PATIENT', columns='DESCRIPTION', values='VALUE').reset_index()
px_df = patients_df.merge(gfr_df, left_on='Id', right_on='PATIENT')

sel_cols = ['Id',
 'BIRTHDATE',
 'DEATHDATE', 
 'RACE',
 'ETHNICITY',
 'GENDER',
 'ZIP',
 'Glomerular filtration rate/1.73 sq M.predicted']

col_names = ['Id',
 'BIRTHDATE',
 'DEATHDATE', 
 'RACE',
 'ETHNICITY',
 'GENDER',
 'ZIP',
 'eGFR']

px_df = px_df[sel_cols]
px_df.columns = col_names
px_df['eGFR'] = px_df.eGFR.astype(float)

### Build CKD Cohorts
Source: https://www.medscape.com/answers/238798-105207/what-are-the-stages-of-chronic-kidney-disease-ckd#:~:text=Stage%201%3A%20Kidney%20damage%20with,%2Fmin%2F1.73%20m%202)
- Stage 1: Kidney damage with normal or increased GFR (>90 mL/min/1.73 m 2)
- Stage 2: Mild reduction in GFR (60-89 mL/min/1.73 m 2)
- Stage 3a: Moderate reduction in GFR (45-59 mL/min/1.73 m 2)
- Stage 3b: Moderate reduction in GFR (30-44 mL/min/1.73 m 2)
- Stage 4: Severe reduction in GFR (15-29 mL/min/1.73 m 2)
- Stage 5: Kidney failure (GFR < 15 mL/min/1.73 m 2 or dialysis)

In [6]:
stage_2 = px_df[px_df.eGFR.between(60, 89)]
stage_3a = px_df[px_df.eGFR.between(45, 59)]
stage_3b = px_df[px_df.eGFR.between(30, 44)]
stage_4 = px_df[px_df.eGFR.between(15, 29)]
stage_5 = px_df[px_df.eGFR.between(0, 14)]

In [7]:
delay_px = pd.concat([stage_2, stage_3a])
plan_px = pd.concat([stage_3b, stage_4])
esrd_px = stage_5

In [8]:
print('Delay: {} patients'.format(delay_px.shape[0]))
print('Plan Transition: {} patients'.format(plan_px.shape[0]))
print('End Stage: {} patients'.format(esrd_px.shape[0]))

Delay: 118 patients
Plan Transition: 40 patients
End Stage: 40 patients


### Estimate Outcomes

In [9]:
esrd_px.head()

,Id,BIRTHDATE,DEATHDATE,RACE,ETHNICITY,GENDER,ZIP,eGFR
15,844d1c39-6ca2-48df-a3de-f9a12633f69c,1963-07-20,NaN,white,nonhispanic,F,NaN,8.9
16,b2b612b0-d4ac-4b83-8d33-4ed6b295c01c,1946-09-23,2012-01-12,white,nonhispanic,F,1887.0,10.9
30,adfc3c2f-30be-430d-8f07-9070dab88c64,1953-03-07,2019-03-23,white,nonhispanic,F,NaN,8.3
34,8a9df121-e012-457c-ade0-ead804d93818,1923-05-15,2001-11-18,white,nonhispanic,F,NaN,11.3
36,efa20606-8972-4bd7-823e-2011fe9a54e1,1923-05-15,1984-12-07,white,nonhispanic,F,1089.0,13.8


### Define Classes

#### Disease

In [11]:
class DiseaseStage(object):
    def __init__(self):
        pass

#### Patient Related

NameError: name 'Careplan' is not defined

#### Plan Related

In [18]:
#### CAREPLANS ####
class Careplan(object):
    def __init__(self,
                 components: list=None,
                 patient=None):
        
        self.components = components
        self.savings = 1
        self.patient = patient
        
    def get_components(self):
        return [comp for comp in self.components]
        
    def get_savings(self):
        for comp in self.components:
            self.savings *= comp.est_savings
        return self.savings
    
            

class DelayPlan(Careplan):
    def __init__(self):
        super().__init__()
        self.components = [InterventionA(),
                           InterventionB()]
        
        self.name = 'Delay'
        
class PrepTransitionPlan(Careplan):
    def __init__(self):
        super().__init__()
        self.components = [InterventionC()]
        
        self.name = 'PrepTransition'
        
class ESRDPlan(Careplan):
    def __init__(self):
        super().__init__()
        self.components = [PalliativeAction1()]
        self.name = 'EndOfLife'


#### PLAN COMPONENTS ####
class Intervention(object):
    def __init__(self,
                 est_savings: float=1,
                 freq='weekly'):
        self.est_savings = est_savings
        self.freq = freq


class DepressionTreatment(Intervention):
    def __init__(self,
                 est_savings: float=.5,
                 freq='daily'):
        super().__init__(est_savings,
                         freq)
        self.desc = 'Depression Treatment'


class Dialysis(Intervention):
    def __init__(self,
                 est_savings: float=.3):
        super().__init__(est_savings)
        self.desc = 'Dialysis'
        
class DietaryProgram(Intervention):
    def __init__(self,
                 est_savings: float=.7):
        super().__init__(est_savings)
        self.desc = 'Dietary Program'
        

class EducationEngagement(Intervention):
    def __init__(self,
                 est_savings: float=1):
        super().__init__(est_savings)
        self.desc = 'Education and Engagement'
        
class WrapAround(Intervention):
    def __init__(self,
                 est_savings: float=1):
        super().__init__(est_savings)
        self.desc = 'High Acuity Wrap-Around Services'
        
class MTM(Intervention):
    def __init__(self,
                 est_savings: float=1):
        super().__init__(est_savings)
        self.desc = 'MTM'
        
class VascularAccess(Intervention):
    def __init__(self,
                 est_savings: float=1):
        super().__init__(est_savings)
        self.desc = 'Vascular Access'

### CKD Stages

In [ ]:
class CKDStage(object):
    def __init__(self):
        pass
    
class Stage1(CKDStage):
    def __init__(self):
        pass
    
class Stage2(CKDStage):
    def __init__(self):
        pass
    
class Stage3A(CKDStage):
    def __init__(self):
        pass
    
class Stage3B(CKDStage):
    def __init__(self):
        pass
    
class Stage4(CKDStage):
    def __init__(self):
        passs
    
class Stage5(CKDStage):
    def __init__(self):
        pass
    

In [15]:
class Patient(object):
    def __init__(self,
                 patient_id: str,
                 eGFR: float,
                 dob: str=None,
                 dod: str=None,
                 ethnicity: str=None,
                 race: str=None,
                 gender: str=None,
                 zipcode: str=None):
        
        self.patient_id = patient_id
        
        self.eGFR = eGFR
        self.assign_stage()
        
        self.careplan = None
        self.low_SES = False

    def assign_stage(self):
        if self.eGFR < 15:
            self.ckd_stage = 5
        elif self.eGFR <= 29:
            self.ckd_stage = 4
        elif self.eGFR <= 44:
            self.ckd_stage = 3.5
        elif self.eGFR <= 59:
            self.ckd_stage = 3
        elif self.eGFR <= 89:
            self.ckd_stage = 2
        
    def generate_careplan(self, plan: Careplan=None):
        if self.ckd_stage <= 3:
            self.careplan = DelayPlan()
        elif self.ckd_stage <= 4:
            self.careplan = PrepTransitionPlan()
        elif self.ckd_stage <= 5:
            self.careplan = ESRDPlan()
            
            
        
    def describe_plan_components(self):
        return [comp.desc for comp in self.careplan.components]   


### Risk Factors

In [23]:
class RiskFactor(object):
    def __init__(self, name):
        self.name = name
    
    
#### BEHAVIORAL RISK FACTORS ######    
class BehavioralRiskFactor(RiskFactor):
    def __init__(self, name):
        super().__init__(name)
        
class Depression(BehavioralRiskFactor):
    def __init__(self, name):
        super().__init__(name)
        
class EngagedWithNephro(BehavioralRiskFactor):
    def __init__(self, name):
        super().__init__(name)
        
class PoorDiet(BehavioralRiskFactor):
    def __init__(self, name):
        super().__init__(name)
        
        
#### DEMOGRAPHIC RISK FACTORS ######    
class DemographicRiskFactor(RiskFactor):
    def __init__(self, name):
        super().__init__(name)
        
class CKDLiterate(DemographicRiskFactor):
    def __init__(self, name):
        super().__init__(name)
        
class Race(DemographicRiskFactor):
    def __init__(self, name):
        super().__init__(name)
        
        
#### PHYSIO RISK FACTORS ######
class PhysioRiskFactor(RiskFactor):
    def __init__(self, name):
        super().__init__(name)
        
class Diabetes(PhysioRiskFactor):
    def __init__(self, name):
        super().__init__(name)

class eGFR(PhysioRiskFactor):
    def __init__(self, name):
        super().__init__(name)

class Hypertension(PhysioRiskFactor):
    def __init__(self, name):
        super().__init__(name)
        
class Malnourished(PhysioRiskFactor):
    def __init__(self, name):
        super().__init__(name)
        
class Obesity(PhysioRiskFactor):
    def __init__(self, name):
        super().__init__(name)
        
class Smoking(PhysioRiskFactor):
    def __init__(self, name):
        super().__init__(name)

#### Testing

In [386]:
stage_2_patient = Patient(patient_id='kdjsn',
             eGFR=70)

stage_5_patient = Patient(patient_id='kdjsn',
             eGFR=13)
iv1 = InterventionA(est_savings=.9)
iv2 = InterventionA(est_savings=.5)
plan1 = Careplan(components=[iv1, iv2])

In [387]:
stage_2_patient.generate_careplan()

In [388]:
stage_2_patient.careplan.get_components()

In [389]:
stage_5_patient.ckd_stage

5

In [390]:
stage_5_patient.generate_careplan()

In [391]:
stage_5_patient.careplan.get_components()

In [392]:
IDX = 10
tmp_patient = Patient(patient_id=px_df.iloc[IDX]['Id'],
                      eGFR=px_df.iloc[IDX]['eGFR'])
print(tmp_patient.ckd_stage)

2


In [393]:
tmp_patient.generate_careplan()

In [394]:
tmp_patient_plan_components = tmp_patient.careplan.get_components()
print(tmp_patient.careplan.name)
for comp in tmp_patient_plan_components:
    print(comp.desc)

Delay
Intervention A
Intervention B
